<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/recipe1M-parser/data/recipe1M/parser/parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recipe1M parser

In [1]:
!pip install quantulum3
!pip install stemming
!pip install pattern

     |████████████████████████████████| 10.7 MB 4.7 MB/s 
     |████████████████████████████████| 101 kB 8.8 MB/s 
  Created wheel for stemming: filename=stemming-1.0.1-py3-none-any.whl size=11138 sha256=f56a673f2b3379b7fd7d00634f859050cce0b887845bf2ab8f8ea743b23c92a2
  Stored in directory: /root/.cache/pip/wheels/6b/e5/e2/c52ebc0a5b53fd82b00cc385e57bb1c90bd50e5f54ddbc06d1
Successfully built stemming
     |████████████████████████████████| 22.2 MB 56.1 MB/s 
     |████████████████████████████████| 87 kB 6.5 MB/s 
     |████████████████████████████████| 81 kB 7.4 MB/s 
     |████████████████████████████████| 5.6 MB 41.5 MB/s 
     |████████████████████████████████| 5.6 MB 45.2 MB/s 
     |████████████████████████████████| 419 kB 57.5 MB/s 
     |████████████████████████████████| 104 kB 69.7 MB/s 
     |████████████████████████████████| 3.6 MB 44.0 MB/s 
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332721 sha256=51ad20a77638556ce131f89937a0d85d20c4bb840e398fc

In [2]:
# Add GDrive
from google.colab import drive
import sys
drive.mount('/content/drive/')
sys.path.append('/content/drive/My Drive/Datasets/Recipe1M/')

Mounted at /content/drive/


In [1]:
import pandas as pd
import re
from recipe import Recipe

#FILE_DIR = '../'
FILE_DIR = '/content/drive/My Drive/Datasets/Recipe1M/'

## Recipe1M data
Recipe1M comes with various json files containing crawled recipes from the web. For our project, two of them are interesting:
* layer1.json: Contains all recipes to their full extend
  
  ![layer1](https://github.com/mscholl96/mad-recime/blob/recipe1M-parser/data/recipe1M/dataset-analysis/layer1_puml.png?raw=1)

* det_ingrs.json: Only contains recipe ID, parsed ingredients and validity flag for parsing 
  
  ![det_ingrs](https://github.com/mscholl96/mad-recime/blob/recipe1M-parser/data/recipe1M/dataset-analysis/det_ingrs_puml.png?raw=1)

In our first attempt we want to make use of the parsed ingredient list and only consider recipes, where all ingredients are marked valid. The parsed ingredients don't contain amounts, so our parser has to kind of merge content of both files. Extracting ingredients from one and their amount and unit from the other file.

## Preprocessing
Removal of all invalid sets from ingredient and full data json to reduce memory. Use pickle instead of json.

In [2]:
ingredient_data = None
recipe_data = None
ingredient_file = FILE_DIR + 'det_ingrs.json'
layer1_file = FILE_DIR + 'layer1.json'

ingredient_out = FILE_DIR + '2022_01_29/det_ingrs_valid.pkl'
layer1_out = FILE_DIR + '2022_01_29/layer1_valid.pkl'

# Get data, set id as used index and drop unnecessary information
ingredient_data = pd.read_json(ingredient_file).set_index('id')
recipe_data = pd.read_json(layer1_file).drop(columns=['url', 'partition']).set_index('id')

# Drop recipes with more than 20 ingredients
indices = ingredient_data[[True if len(row) > 20 else False for row in ingredient_data['valid']]].index
ingredient_data = ingredient_data.drop(indices)
recipe_data = recipe_data.drop(indices)
print(f'Removed {len(indices)} recipes with more than 20 ingredients')

# Drop recipes with more than 30 instructions
indices = recipe_data[[True if len(row) > 30 else False for row in recipe_data['instructions']]].index
ingredient_data = ingredient_data.drop(indices)
recipe_data = recipe_data.drop(indices)
print(f'Removed {len(indices)} recipes with more than 30 instructions')

# Removal of all elements in ingredient json which contain invalid entries according to the data set
# Get indices of ingredients which contain false valid flags 
indices = ingredient_data[[True if any(x == False for x in row) else False for row in ingredient_data['valid']]].index
ingredient_data = ingredient_data.drop(indices).drop(columns=['valid'])
recipe_data = recipe_data.drop(indices)
print(f'Removed {len(indices)} recipes with invalid ingredients')

indices = []
# Replace fractions in raw ingredients
fractionRegex = re.compile('[0-9]+/[0-9]+')
# Replace patterns like "1 12" which is intended to be 1 1/2 (1.5)
doubleRegex = re.compile('[1-9] [1-9][2-9]')

# Bring both datasets together
recipe_data['ingredients_parsed'] = ingredient_data['ingredients']

for idx, recipe in recipe_data.iterrows():
    # Put every information into one data set
    for ingredient in recipe['ingredients']:
        text = ingredient['text']
        if text:
            fractions = re.findall(fractionRegex, text)
            doubleNumbers = re.findall(doubleRegex, text)
            if fractions:
                numbers = fractions[0].split('/')
                float_representation = int(numbers[0])/int(numbers[1])
                text = text.replace(fractions[0], str(float_representation))
                ingredient['text'] = text
            elif doubleNumbers:
                numbers = doubleNumbers[0].split(' ')
                float_representation = int(numbers[0]) + int(numbers[1][0])/int(numbers[1][1])
                text = text.replace(doubleNumbers[0], str(float_representation))
                ingredient['text'] = text
        elif idx not in indices:
            # To be removed later
            indices.append(idx)



# Remove empty ingredients from data
for index in indices:
    recipe_data.loc[index]['ingredients'] = [elem for elem in recipe_data.loc[index]['ingredients'] if elem['text']]
    recipe_data.loc[index]['ingredients_parsed'] = [elem for elem in recipe_data.loc[index]['ingredients_parsed'] if elem['text']]

# Save data to pickle (it's faster)
recipe_data.to_pickle(layer1_out)

recipe_data.head(10)



Removed 18626 recipes with more than 20 ingredients
Removed 14270 recipes with more than 30 instructions
Removed 149732 recipes with invalid ingredients


,ingredients,title,instructions,ingredients_parsed
id,,,,
000033e39b,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",Dilly Macaroni Salad Recipe,[{'text': 'Cook macaroni according to package ...,"[{'text': 'elbow macaroni'}, {'text': 'America..."
000035f7ed,"[{'text': '8 tomatoes, quartered'}, {'text': '...",Gazpacho,[{'text': 'Add the tomatoes to a food processo...,"[{'text': 'tomatoes'}, {'text': 'kosher salt'}..."
00003a70b1,"[{'text': '2.5 cups milk'}, {'text': '1.5 cups...",Crunchy Onion Potato Bake,[{'text': 'Preheat oven to 350 degrees Fahrenh...,"[{'text': 'milk'}, {'text': 'water'}, {'text':..."
00004320bb,[{'text': '1 (3 ounce) package watermelon gela...,Cool 'n Easy Creamy Watermelon Pie,"[{'text': 'Dissolve Jello in boiling water.'},...","[{'text': 'watermelon gelatin'}, {'text': 'boi..."
0000631d90,"[{'text': '12 cup shredded coconut'}, {'text':...",Easy Tropical Beef Skillet,"[{'text': 'In a large skillet, toast the cocon...","[{'text': 'shredded coconut'}, {'text': 'lean ..."
000075604a,"[{'text': '2 Chicken thighs'}, {'text': '2 tsp...",Kombu Tea Grilled Chicken Thigh,[{'text': 'Pierce the skin of the chicken with...,"[{'text': 'chicken thighs'}, {'text': 'tea'}, ..."
00007bfd16,"[{'text': '6 -8 cups fresh rhubarb, or'}, {'te...",Strawberry Rhubarb Dump Cake,[{'text': 'Put ingredients in a buttered 9 x 1...,"[{'text': 'fresh rhubarb'}, {'text': 'frozen r..."
000095fc1d,"[{'text': '8 ounces, weight Light Fat Free Van...",Yogurt Parfaits,[{'text': 'Layer all ingredients in a serving ...,"[{'text': 'non - fat vanilla yogurt'}, {'text'..."
0000973574,"[{'text': '2 cups flour'}, {'text': '1 tablesp...",Zucchini Nut Bread,"[{'text': 'Sift dry ingredients.'}, {'text': '...","[{'text': 'flour'}, {'text': 'cinnamon'}, {'te..."


## Actual parsing

In [2]:
recipes = []
# Sort by ID, 
recipe_data = pd.read_pickle(FILE_DIR + '2022_01_29/layer1_valid.pkl')
print(f'Total number of recipes: {len(recipe_data)}')
recipe_data.head(5)


Total number of recipes: 847092


,ingredients,title,instructions,ingredients_parsed
id,,,,
000033e39b,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",Dilly Macaroni Salad Recipe,[{'text': 'Cook macaroni according to package ...,"[{'text': 'elbow macaroni'}, {'text': 'America..."
000035f7ed,"[{'text': '8 tomatoes, quartered'}, {'text': '...",Gazpacho,[{'text': 'Add the tomatoes to a food processo...,"[{'text': 'tomatoes'}, {'text': 'kosher salt'}..."
00003a70b1,"[{'text': '2.5 cups milk'}, {'text': '1.5 cups...",Crunchy Onion Potato Bake,[{'text': 'Preheat oven to 350 degrees Fahrenh...,"[{'text': 'milk'}, {'text': 'water'}, {'text':..."
00004320bb,[{'text': '1 (3 ounce) package watermelon gela...,Cool 'n Easy Creamy Watermelon Pie,"[{'text': 'Dissolve Jello in boiling water.'},...","[{'text': 'watermelon gelatin'}, {'text': 'boi..."
0000631d90,"[{'text': '12 cup shredded coconut'}, {'text':...",Easy Tropical Beef Skillet,"[{'text': 'In a large skillet, toast the cocon...","[{'text': 'shredded coconut'}, {'text': 'lean ..."


In [ ]:
j = 0
for idx, raw_recipe in recipe_data.iterrows():
    
    recipe = Recipe(idx, raw_recipe['title'])
    
    recipe.parse_ingredients(raw_recipe['ingredients_parsed'])

    recipe.parse_instructions(raw_recipe['instructions'])
      
    recipe.get_ingredient_amounts(raw_recipe['ingredients'])

    recipes.append(recipe)

    j += 1
    if j % 5000 == 0:
      print(f'Progress: {j}')
      # Save every 100000 recipes
      if j % 100000 == 0:
        print(f'Parsed {j} recipes')
        # Create data frame in the end (according to Stackoverflow this is faster)                
        df = pd.DataFrame([r.to_dict() for r in recipes]).set_index('id')
        path = FILE_DIR + f'2022_01_29/recipes_valid_{int(j / 100000)}'
        df.to_pickle(path + '.pkl')
        del df
        recipes = []


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your ow

Progress: 5000


In [ ]:
# Try to clean up
del recipe_data

# Create data frame in the end (according to Stackoverflow this is faster)                
df = pd.DataFrame([r.to_dict() for r in recipes]).set_index('id')
del recipes

path = FILE_DIR + '2022_01_29/recipes_valid'

df.to_pickle(path + '.pkl')
df.to_json(path + '.json', indent=2, orient='records')
df.head(10)

,title,ingredients,instructions
id,,,
000033e39b,Dilly Macaroni Salad Recipe,amount unit ingredient 0 1....,0 Cook macaroni according to package direct...
000035f7ed,Gazpacho,amount unit ingredient 0 8.0 ...,0 Add the tomatoes to a food processor with...
00003a70b1,Crunchy Onion Potato Bake,amount unit ingredient 0 1....,0 Preheat oven to 350 degrees Fah...
00004320bb,Cool 'n Easy Creamy Watermelon Pie,amount unit ingredient 0 1.0...,0 Dissolve Jello in boiling water. 1 ...
0000631d90,Easy Tropical Beef Skillet,amount unit ingredient 0...,"0 In a large skillet, toast the coconut ove..."
000075604a,Kombu Tea Grilled Chicken Thigh,amount unit ingredient 0 2.0 ...,0 Pierce the skin of the chicken with a for...
00007bfd16,Strawberry Rhubarb Dump Cake,amount unit ing...,0 Put ingredients in a buttered 9 x 12 x 2-...
000095fc1d,Yogurt Parfaits,amount unit ingredient 0 ...,0 Layer all ingredients in a serving dish. ...
0000973574,Zucchini Nut Bread,amount unit ingredient 0 2...,0 Sift dry ingr...
